In [48]:

import pandas as pd


from typing import List, Dict, Tuple
import random
import pickle
import pprint

from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

from surprise.similarities import cosine, msd, pearson, pearson_baseline

product_data = {
    "data_context": "books",
    "product_filepath": "data/products_books_v1_10_10.csv",
    "transactions_filepath": "data/transactions_books_v1_10_10.csv",
    "features": ["product_title", "product_image", "product_soup", "product_images"],
    "version": "1.0",
    "unique_name": "_books_v1_10_10",
}

print("looking at", "../" + product_data["product_filepath"])

productdf =  pd.read_csv("../" + product_data["product_filepath"])
transactiondf = pd.read_csv("../" + product_data["transactions_filepath"])


print(len(transactiondf))
productdf.head()
transactiondf.head()


looking at ../data/products_books_v1_10_10.csv
381082


,id,user_id,product_id,rate
0,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,0446364193,0
1,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3379015180,0
2,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3404148576,4
3,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3423071516,5
4,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3442413508,5


In [15]:

class RecommendationAbstract():
    strategy_name: str = "REQUIRES IMPLEMENTATION"
    version: str = "REQUIRES IMPLEMENTATION"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = "REQUIRES IMPLEMENTATION"
    supports_past_recommendation: bool = "REQUIRES IMPLEMENTATION"

    def __init__(self, products, product_data):
        self.products = products
        self.product_data = product_data
        self.model = None
        # populate id_to_products
        self.id_to_products = {}
        for product in self.products.to_dict(orient='records'):
            self.id_to_products[product['id']] = product

    def loadModel(self, model_code):
        """
        Load the model
        """
        self.model = model_code

    def train(self, verbose=False, transactions_train=None, users_train=None):
        """
        Train the model
        """
        # ... do training
        # self.model = trained_model
        
    def get_random_recommendation(self, n=1):
        """
        Get random recommendations
        """
        # Select n random rows from the DataFrame
        random_rows = self.products.sample(n)
        # Convert the selected rows to a list of dictionaries
        random_recommendations = random_rows.to_dict(orient='records')
        return random_recommendations



    def saveModel(self, model_code):
        """
        Save the model
        """
        # ... saves the model

    def id_to_productDetail(self, product_id: str) -> Dict[str, str]:
        """
        Return product details based on product id.
        """
        return self.id_to_products.get(product_id)

    def ids_to_products(self, ids: List[str]) -> List[Dict[str, str]]:
        """
        Return product details for a list of product ids.
        """
        return [self.id_to_productDetail(id) for id in ids]

    def like(self, keyword: str) -> List[str]:
        """
        Return a list of products that contain the given keyword in their title.
        """
        return [product for product in self.products if keyword in product['product_title']]

    def recommend_from_single(self, product_id: str, n=5) -> List[str]:
        """
        Return recommendations based on a single product.
        """
        target_name = self.id_to_productDetail(product_id)['product_title']
        keywords = target_name.split(" ")
        recommendations = []
        for keyword in keywords:
            recommendations.extend(self.like(keyword))
        
        random.shuffle(recommendations)
        return recommendations[:n]

    def recommend_from_past(self, user_transactions, n=10) -> List[str]:
        """
        Return recommendations based on past user transactions.
        """
        rec = []
        for transaction in user_transactions:
            rec.extend(self.recommend_from_single(transaction['product_id']))
        random.shuffle(rec)
        return rec[:n]

In [57]:

class SimilutudeRecommender(RecommendationAbstract):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Similitude Recommender"
    slug_name: str = "simmilitude_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def __init__(self, products: pd.DataFrame, product_data: dict, transactions = None):
        super().__init__(products, product_data)
        self.products = products
        self.model = None
        
        # Get the product ids and store them.
        self.product_ids = self.products['id'].unique()
        self.all_transactions_df = transactions
        
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "pearson", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        
        reader = Reader(rating_scale=(1, 5))
        
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    def get_filename(self):
        return "models/" + self.slug_name + self.product_data["unique_name"] + ".pik"
    
    def save(self):
        # Store self.pt
        filename = self.get_filename()
        model_file = open(filename, 'wb')
        pickle.dump(self.model, model_file)
        model_file.close()
        
    def load(self, auto_create=True):
        
        filename = self.get_filename()
        try:
            model_file = open(filename, 'rb')
            self.model = pickle.load(model_file)
            model_file.close()
        except:
            self.save()
            
    def getNeighbors(self, product_id: str, n=5):
        """
        Returns the neighbors of a product
        """
        product_inner_id = self.model.trainset.to_inner_iid(product_id)
        neighbors = self.model.get_neighbors(product_inner_id, k=n)
        return neighbors

    def recommend_from_single(self, product_id: str, n=5) -> List[Tuple[dict, float]]:
        """
        
        # To optimize things, SVD takes a Similitude type recommender. Which posses the method (receive product neighbors.)
        """
        recommendation_list: List[tuple[dict, float]] = []
        product_inner_id = self.model.trainset.to_inner_iid(product_id)
        neighbors = self.model.get_neighbors(product_inner_id, k=n)
        
        # for each neighbor, try to predict and prioritize given a user in all_transactions_that shared that book as well.
        for neighbor_book_inner_id in neighbors:
            # get user_id that top rated the product sort the relevant_transactions
            if neighbor_book_inner_id == product_inner_id:
                continue
            
            product_serie = self.products.iloc[neighbor_book_inner_id]
            neighbor_book_id = product_serie['id']
            product = self.id_to_products[neighbor_book_id]
            
            if product['product_id'] == product_id:
                continue
            
            recommendation_list.append((self.id_to_products[neighbor_book_id], 1))
        
        # sort recommendations
        random.shuffle(recommendation_list)
        return recommendation_list[:n]
        

    def recommend_from_past(self, transactions: List[str], n=10):
        """
        Calls for each transaction the recommend_from_single method.
        Gives Priority if seen multiple recommendations.
        Shuffle and returns :n
        """
        recs = set()
        recs_seen_times = {}
        products_dictionary = {}
        
        # Deprecated.
        # if(len(transactions) > 2):
        #     return self.collaborativestore_predict_population(
        #         transactions, n=n
        #     )
        
        for transaction in transactions:
            recs = self.recommend_from_single(transaction)
            for rec_id, confidence in recs:
                
                if rec_id in recs:
                    recs_seen_times[rec_id['id']] += 1
                else:
                    products_dictionary[rec_id['id']] = rec_id
                    recs_seen_times[rec_id['id']] = 1
        
        for rec_id in recs_seen_times:
            recs.append((products_dictionary[rec_id], recs_seen_times[rec_id]))
            
        recs = list(recs)
        
        recs.sort(key=lambda x: x[1], reverse=True)
        return recs
    

In [58]:

engineRec = SimilutudeRecommender(productdf, product_data, transactions=transactiondf)
# engineRec.train(transactions=transactiondf, auto_save=True)
engineRec.load()

  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
# SEARCH_TEST = engineRec.recommend_from_single(randomProduct['id'])
# pprint.pprint(rec)
# 

print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
# pprint.pprint(engineRec.product_ids[SEARCH_TEST])
rec = engineRec.recommend_from_single(SEARCH_TEST)
pprint.pprint(rec)


{'count': 13,
 'id': '0140088946',
 'product_id': '0140088946',
 'product_image': 'http://images.amazon.com/images/P/0140088946.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'Less Than Zero (Contemporary American Fiction) Bret Easton '
                 'Ellis Penguin Books',
 'product_tags': nan,
 'product_title': 'Less Than Zero (Contemporary American Fiction)'}
======== RECOMENDATIONS SINGLE CASE =========== 
[({'count': 131,
   'id': '044651862X',
   'product_id': '044651862X',
   'product_image': 'http://images.amazon.com/images/P/044651862X.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Celestine Prophecy (Celestine Prophecy) James Redfield '
                   'Warner Books',
   'product_tags': nan,
   'product_title': 'The Celestine Prophecy (Celestine Prophecy)'},
  1),
 ({'count': 200,
   'id': '0449212602',
   'product_id': '0449212602',
   'product_image': 'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg',
   'product_price': nan,

In [59]:
  
  

# randomProduct = engineRec.get_random_recommendation()[0]
# pprint.pprint(randomProduct)

# rec = engineRec.recommend_from_single(randomProduct['id'])
# pprint.pprint(rec)
# 

SEARCH_TEST = '0590353403'

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
print('======== RECOMENDATIONS SINGLE CASE =========== ')
pprint.pprint(rec)


{'count': 122,
 'id': '0590353403',
 'product_id': '0590353403',
 'product_image': 'http://images.amazon.com/images/P/0590353403.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': "Harry Potter and the Sorcerer's Stone (Book 1) J. K. Rowling "
                 'Scholastic',
 'product_tags': nan,
 'product_title': "Harry Potter and the Sorcerer's Stone (Book 1)"}
---------
======== RECOMENDATIONS SINGLE CASE =========== 
[({'count': 131,
   'id': '044651862X',
   'product_id': '044651862X',
   'product_image': 'http://images.amazon.com/images/P/044651862X.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Celestine Prophecy (Celestine Prophecy) James Redfield '
                   'Warner Books',
   'product_tags': nan,
   'product_title': 'The Celestine Prophecy (Celestine Prophecy)'},
  1),
 ({'count': 200,
   'id': '0449212602',
   'product_id': '0449212602',
   'product_image': 'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg',
   'product_price': 

In [60]:


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)


=============  RECOMENDATIONS RECOMMENDATIONS  ============
[({'count': 207,
   'id': '0446608955',
   'product_id': '0446608955',
   'product_image': 'http://images.amazon.com/images/P/0446608955.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'A Walk to Remember Nicholas Sparks Warner Books',
   'product_tags': nan,
   'product_title': 'A Walk to Remember'},
  1),
 ({'count': 172,
   'id': '0684874350',
   'product_id': '0684874350',
   'product_image': 'http://images.amazon.com/images/P/0684874350.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': "ANGELA'S ASHES Frank McCourt Scribner",
   'product_tags': nan,
   'product_title': "ANGELA'S ASHES"},
  1),
 ({'count': 178,
   'id': '0425147622',
   'product_id': '0425147622',
   'product_image': 'http://images.amazon.com/images/P/0425147622.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Body Farm Patricia Daniels Cornwell Berkley Publishing '
                   'Group',
   'product_tags': 

In [61]:

class PearsonSimilitude(SimilutudeRecommender):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Pearson Recommender"
    slug_name: str = "simmilitude_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "pearson", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    

In [62]:

engineRec = SimilutudeRecommender(productdf, product_data, transactions=transactiondf)
# engineRec.train(transactions=transactiondf, auto_save=True)
engineRec.load()


randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']

print('======== RECOMENDATIONS SINGLE CASE =========== ')
rec = engineRec.recommend_from_single(SEARCH_TEST)
pprint.pprint(rec)


print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)


{'count': 10,
 'id': '037371128X',
 'product_id': '037371128X',
 'product_image': 'http://images.amazon.com/images/P/037371128X.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'The Secret Daughter  (Raising Cane) Roz Denny Fox Harlequin',
 'product_tags': nan,
 'product_title': 'The Secret Daughter  (Raising Cane)'}
======== RECOMENDATIONS SINGLE CASE =========== 
[({'count': 77,
   'id': '0451186923',
   'product_id': '0451186923',
   'product_image': 'http://images.amazon.com/images/P/0451186923.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Deep End of the Ocean Jacquelyn Mitchard New Amer '
                   'Library',
   'product_tags': nan,
   'product_title': 'The Deep End of the Ocean'},
  1),
 ({'count': 130,
   'id': '038550120X',
   'product_id': '038550120X',
   'product_image': 'http://images.amazon.com/images/P/038550120X.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'A Painted House JOHN GRISHAM Doubleday',
   'product_tags'

In [63]:

class MeanSquaredSimilitude(SimilutudeRecommender):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Mean Squared Recommender"
    slug_name: str = "mean_squared_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "msd", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    

In [64]:

engineRec = MeanSquaredSimilitude(productdf, product_data, transactions=transactiondf)
# engineRec.train(transactions=transactiondf, auto_save=True)
engineRec.load()


randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']

print('======== RECOMENDATIONS SINGLE CASE =========== ')
rec = engineRec.recommend_from_single(SEARCH_TEST)
pprint.pprint(rec)


print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)


{'count': 8,
 'id': '3423126949',
 'product_id': '3423126949',
 'product_image': 'http://images.amazon.com/images/P/3423126949.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'Die groÃ?Â?e Umwendung. Neue Briefe in die chinesische '
                 'Vergangenheit. Herbert Rosendorfer Dtv',
 'product_tags': nan,
 'product_title': 'Die groÃ?Â?e Umwendung. Neue Briefe in die chinesische '
                  'Vergangenheit.'}
======== RECOMENDATIONS SINGLE CASE =========== 
[({'count': 252,
   'id': '0440224764',
   'product_id': '0440224764',
   'product_image': 'http://images.amazon.com/images/P/0440224764.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Partner John Grisham Dell Publishing Company',
   'product_tags': nan,
   'product_title': 'The Partner'},
  1),
 ({'count': 270,
   'id': '0446610038',
   'product_id': '0446610038',
   'product_image': 'http://images.amazon.com/images/P/0446610038.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup':

In [65]:

class PearsonBaselineSimilitude(SimilutudeRecommender):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Pearson Baseline Recommender"
    slug_name: str = "pearson_baseline"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "pearson_baseline", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    

In [66]:

engineRec = PearsonBaselineSimilitude(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()

print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
=============  RECOMENDATIONS RECOMMENDATIONS  ============
[({'count': 238,
   'id': '0316096199',
   'product_id': '0316096199',
   'product_image': 'http://images.amazon.com/images/P/0316096199.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Lucky : A Memoir Alice Sebold Back Bay Books',
   'product_tags': nan,
   'product_title': 'Lucky : A Memoir'},
  1),
 ({'count': 266,
   'id': '0446606812',
   'product_id': '0446606812',
   'product_image': 'http://images.amazon.com/images/P/0446606812.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Message in a Bottle Nicholas Sparks Warner Vision',
   'product_tags': nan,
   'product_title': 'Message in a Bottle'},
  1),
 ({'count': 40,
   'id': '0786884142',
   'product_id': '0786884142',
   'product_image': 'http://images.amazon.com/images/P/0786884142.01.MZZZZZZZ.jpg',
   'product_price': nan,


In [67]:

class PearsonSimilitude(SimilutudeRecommender):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Pearson Recommender"
    slug_name: str = "pearson"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
        
        
    

In [68]:

class CosineSimilitude(SimilutudeRecommender):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Cosine Recommender"
    slug_name: str = "cosine"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "cosine", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    

In [69]:

engineRec = PearsonBaselineSimilitude(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()

print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
=============  RECOMENDATIONS RECOMMENDATIONS  ============
[({'count': 39,
   'id': '0385323638',
   'product_id': '0385323638',
   'product_image': 'http://images.amazon.com/images/P/0385323638.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Pink Slip RITA CIRESI Delta',
   'product_tags': nan,
   'product_title': 'Pink Slip'},
  1),
 ({'count': 266,
   'id': '0446606812',
   'product_id': '0446606812',
   'product_image': 'http://images.amazon.com/images/P/0446606812.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Message in a Bottle Nicholas Sparks Warner Vision',
   'product_tags': nan,
   'product_title': 'Message in a Bottle'},
  1),
 ({'count': 40,
   'id': '0786884142',
   'product_id': '0786884142',
   'product_image': 'http://images.amazon.com/images/P/0786884142.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'A Mon